<a href="https://colab.research.google.com/github/shubhm253/AI/blob/master/70271018106.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
#ANSWER 1 

E) All of these 
(All of the techniques can be used for the purpose of engineering features in a model)

#ANSWER 2

D) Alpha: density of topics generated within documents, beta: density of terms generated within topics

#ANSWER 3

D) 7, 4, 2
(Nouns: I, New, Delhi, Analytics, Vidhya, Delhi, Hackathon (7)

Verbs: am, planning, visit, attend (4)

Words with frequency counts > 1: to, Delhi (2)

Hence option D is correct.)

#ANSWER 4

B) K * Log(3) / T
(formula for TF is K/T

formula for IDF is log(total docs / no of docs containing “data”)

= log(1 / (⅓))

= log (3)

Hence correct choice is Klog(3)/T)

#ANSWER 5

C) d2 and d4
(Both of the documents d2 and d4 contains 4 terms and does not contain the least number of terms which is 3.)

#ANSWER 6

A) t4, t6

#ANSWER 7

B) t3 – 3/6

"""



'\n#ANSWER 1 \n\nE) All of these \n(All of the techniques can be used for the purpose of engineering features in a model)\n\n#ANSWER 2\n\nD) Alpha: density of topics generated within documents, beta: density of terms generated within topics\n\n#ANSWER 3\n\nD) 7, 4, 2\n(Nouns: I, New, Delhi, Analytics, Vidhya, Delhi, Hackathon (7)\n\nVerbs: am, planning, visit, attend (4)\n\nWords with frequency counts > 1: to, Delhi (2)\n\nHence option D is correct.)\n\n#ANSWER 4\n\nB) K * Log(3) / T\n(formula for TF is K/T\n\nformula for IDF is log(total docs / no of docs containing “data”)\n\n= log(1 / (⅓))\n\n= log (3)\n\nHence correct choice is Klog(3)/T)\n\n#ANSWER 5\n\nC) d2 and d4\n(Both of the documents d2 and d4 contains 4 terms and does not contain the least number of terms which is 3.)\n\n#ANSWER 6\n\nA) t4, t6\n\n#ANSWER 7\n\nB) t3 – 3/6\n\n'

Import the required libraries for LDA and NMF.
We will be using Gensim to do LDA and sklearn for NMF.

In [0]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn
import sys
from nltk.corpus import stopwords
import nltk
from gensim.models import ldamodel
import gensim.corpora
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize
import pickle

Upload the required file on google colab.

In [6]:
from google.colab import files
uploaded = files.upload()

Saving scotch_review.csv to scotch_review.csv


Read the uploaded csv file.

In [0]:
dataset = pd.read_csv('scotch_review.csv')

Overview of the dataset

In [8]:
dataset.head(5)

,Unnamed: 0,name,category,review.point,price,currency,description
0,1,"Johnnie Walker Blue Label, 40%",Blended Scotch Whisky,97,225,$,"Magnificently powerful and intense. Caramels, ..."
1,2,"Black Bowmore, 1964 vintage, 42 year old, 40.5%",Single Malt Scotch,97,4500.00,$,What impresses me most is how this whisky evol...
2,3,"Bowmore 46 year old (distilled 1964), 42.9%",Single Malt Scotch,97,13500.00,$,There have been some legendary Bowmores from t...
3,4,"Compass Box The General, 53.4%",Blended Malt Scotch Whisky,96,325,$,With a name inspired by a 1926 Buster Keaton m...
4,5,"Chivas Regal Ultis, 40%",Blended Malt Scotch Whisky,96,160,$,"Captivating, enticing, and wonderfully charmin..."


We only need the description text column from the data

In [0]:
data_text = dataset[['description']];

Check for null values if there are any.

In [10]:
data_text.isnull().sum()

description    0
dtype: int64

Import stopwords

In [13]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Remove stopwords.

In [0]:
for idx in range(len(data_text)):
  data_text.iloc[idx]['description'] = [word for word in data_text.iloc[idx]['description'].split(' ') if word not in stopwords.words()];
    

In [0]:
if idx % 1000 == 0:
  sys.stdout.write('\rc = ' + str(idx) + ' / ' + str(len(data_text)));

save data in a pickle file for further usage.

In [0]:
pickle.dump(data_text, open('data_text.dat', 'wb'))

Get the words as an array for lda input

In [0]:
train_description = [value[0] for value in data_text.iloc[0:].values];

Number of topics we will cluster for: 5

In [0]:
num_topics = 5;

We will use the gensim library for LDA. First, we obtain a id-2-word dictionary. For each headline, we will use the dictionary to obtain a mapping of the word id to their word counts. The LDA model uses both of these mappings.

In [0]:
id2word = gensim.corpora.Dictionary(train_description)
corpus = [id2word.doc2bow(text) for text in train_description]
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)

We will iterate over the number of topics, get the top words in each cluster, and add them to a DataFrame, than print these words.

In [0]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 20);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict);

Call the function and obtain the topics.

In [21]:
get_lda_topics(lda, num_topics)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,The,The,The,The,whisky
1,A,palate,This,A,A
2,notes,old,year,notes,The
3,finish.,A,I,whisky,notes
4,\r\n,year,whisky,palate,little
5,whisky,finish,palate,nose,smoke
6,I,nose,notes,finish.,sweet
7,nose,This,A,This,like
8,sweet,"palate,",old,sweet,finish.
9,"palate,",sherry,like,light,I


NMF

the count vectorizer needs string inputs, not array, so I join them with a space.

In [0]:
train_description_sentences = [' '.join(text) for text in train_description]

Obtain a Counts design matrix. Because the size of the matrix will be large, we can set the max_features to 5000.

In [0]:
vectorizer = CountVectorizer(analyzer='word', max_features=5000);
x_counts = vectorizer.fit_transform(train_description_sentences);

set a TfIdf transformer, and transfer the counts with the model.

In [0]:
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

Normalize the TfIdf values so each row has unit length.

In [0]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)

Obtain a NMF model.

In [0]:
model = NMF(n_components=num_topics, init='nndsvd');

Fit the model

In [27]:
model.fit(xtfidf_norm)

NMF(alpha=0.0, beta_loss='frobenius', init='nndsvd', l1_ratio=0.0, max_iter=200,
    n_components=5, random_state=None, shuffle=False, solver='cd', tol=0.0001,
    verbose=0)

Generating NMF topics:
We are going to iterate over each topic, obtain the most important scoring words in each cluster, and add them to a Dataframe, as before.

In [0]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

Call the function and obtain the topics

In [29]:
get_nmf_topics(model, 20)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,sherry,required,color,smoke,whisky
1,nose,grandfather,aroma,peat,notes
2,the,kills,gold,the,old
3,chocolate,stirs,body,water,vanilla
4,oak,envelope,fruit,it,year
5,matured,books,flavors,there,fruit
6,casks,port,notes,like,oak
7,finish,meet,vanilla,little,clean
8,spicy,wooded,aromas,fruits,spice
9,palate,lacking,dry,palate,honeyed
